In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec
from lark.data import *

1.8.1
0.8.0a0+e4e171a


In [3]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [4]:
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [5]:
cfg = Config(
    n_workers=18,
    
    n_fft=3200,
    window_length=3200,
    n_mels=128,
    hop_length=800,
    
    use_pink_noise=0,
    use_recorded_noise=0,
    use_overlays=False,
    apply_filter=0,
    sites=['COR'], 
    use_neptune=False, 
    log_batch_metrics=False,
    n_epochs=1000, 
    bs=32,
    lr=1e-3, 
    model='resnest50',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [6]:
cfg.as_dict()

{'sites': ['COR'],
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 18,
 'train_duration': 5,
 'valid_duration': 5,
 'seed': 231,
 'use_pink_noise': 0,
 'use_recorded_noise': 0,
 'use_secondary_labels': False,
 'noise_nsr_dbs': [20, 10, 3],
 'pink_noise_nsr_dbs': [10, 5, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': False,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'apply_filter': 0,
 'use_neptune': False,
 'log_batch_metrics': False,
 'sr': 32000,
 'n_fft': 3200,
 'window_length': 3200,
 'n_mels': 128,
 'hop_length': 800,
 'f_min': 150,
 'f_max': 15000,
 'f1_threshold': 0.5,
 'lr': 0.001,
 'n_epochs': 1000,
 'model': 'resnest50',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels': ['bobfly1',
  'brnjay',
  'bucmot2',
  

In [7]:
cfg.training_dataset_size

2000

In [10]:
prep = Sig2Spec(cfg)
main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)


for param in main_model.parameters():
    param.requires_grad = False

for layer in [main_model.layer3, main_model.layer4, main_model.avgpool]:
    for param in layer.parameters():
        param.requires_grad = True

posp = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
    torch.nn.Dropout(p=0.2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=1024, out_features=512, bias=True),
    torch.nn.Dropout(p=0.2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=512, out_features=len(cfg.labels), bias=True),
)
main_model.fc = posp
model = torch.nn.Sequential(prep, main_model)
model = model.cuda()


Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [11]:
model

Sequential(
  (0): Sig2Spec(
    (melspec): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
    (p2db): AmplitudeToDB()
    (tf_norm): Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  )
  (1): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mod

In [33]:
lrn = Learner("resnest50-vanilla", cfg, model)

resnest50-vanilla-20210514-085931


In [34]:
# x, y = next(iter(lrn.tdl))
# model(x.cuda())

In [35]:
lrn.learn()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-14 09:00:04 epoch:   1 train loss: 0.212912 train f1: 0.040167 valid loss: 0.093367 valid f1: 0.009569


  0%|          | 0/63 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-14 09:13:40 epoch:  25 train loss: 0.031666 train f1: 0.895218 valid loss: 0.098697 valid f1: 0.307102


  0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
lrn.evaluate()

In [26]:
lrn.load_checkpoint('best')

{'epoch': 10,
 'valid_loss': 0.07955082545154973,
 'valid_score': 0.4748743772506714}

In [27]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,413,0,23587,0,0.033834
1,0.1,293,21587,2000,120,0.216556
2,0.2,264,22749,838,149,0.348515
3,0.3,233,23147,440,180,0.429098
4,0.4,213,23313,274,200,0.473333
5,0.5,189,23393,194,224,0.474874
6,0.6,172,23442,145,241,0.471233
7,0.7,153,23484,103,260,0.457399
8,0.8,106,23511,76,307,0.356303
9,0.9,62,23554,33,351,0.244094


In [28]:
lrn.learn()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-14 08:52:08 epoch:   1 train loss: 0.031479 train f1: 0.896259 valid loss: 0.077326 valid f1: 0.483029


  0%|          | 0/63 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-14 08:58:59 epoch:  13 train loss: 0.040860 train f1: 0.856687 valid loss: 0.092160 valid f1: 0.304516


  0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 